In [1]:
!pip install transformers
!pip install multimodal-transformers
!pip install -U torch==1.8.0 torchtext==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.4 MB/s 
     |████████████████████████████████| 6.6 MB 37.0 MB/s 
     |████████████████████████████████| 596 kB 62.7 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 16.0 MB/s 
     |████████████████████████████████| 1.2 MB 60.7 MB/s 
     |████████████████████████████████| 3.0 MB 52.6 MB/s 
     |████████████████████████████████| 880 kB 69.5 MB/s 
  Created wheel for multimodal-transformers: filename=multimodal_transformers-0.1.4a0-py3-none-any.whl size=20631 sha256=97bd259213078a685208e3af3ac4bebad7214d95f55507d9a4ebed29ca4122eb
  Sto

In [3]:
from google.colab import drive
import json
import logging
import torch
import os
import re
import numpy as np
import pandas as pd
from typing import Optional
from dataclasses import dataclass, field
from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular
from transformers.training_args import TrainingArguments
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, EvalPrediction, set_seed)
from transformers import TextClassificationPipeline
from transformers import pipeline
from transformers import BertConfig
from transformers import AutoModel

drive.mount('/content/gdrive')
data_path = '/content/gdrive/My Drive/data/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
def clean_tweets(tweets):
    # eliminamos enlaces 
    tweets_no_urls = [re.sub(r'http\S+', '', word) for word in tweets]
    # eliminamos caracteres especiales y signos de puntuacion
    tweets_processed = [re.sub(r"[^a-zA-Z0-9_ÑñÁáÉéÍíÓóÚú!?¿¡]+", ' ', word) for word in tweets_no_urls]

    return tweets_processed

In [27]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
data_df = pd.read_csv(data_path + 'all_csvs/todos_tweets_juntos.csv')
# limpiar texto ?
tweets = data_df['text'].tolist()
tweets_processed = clean_tweets(tweets)
print(len(tweets_processed))
# print(tweets)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,5,7,8,9,11,13,15,21,27,28,31,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


40529


In [28]:
# load model from file saved previously
pretrained_model = AutoModelForSequenceClassification.from_pretrained(data_path + "models_saved/SocialHaterBERT/")
# load tokenizer from the model
tokenizer = AutoTokenizer.from_pretrained(data_path + "models_saved/SocialHaterBERT/")

Some weights of the model checkpoint at /content/gdrive/My Drive/data/models_saved/SocialHaterBERT/ were not used when initializing BertForSequenceClassification: ['tabular_combiner.h_bias', 'tabular_combiner.num_bn.weight', 'tabular_combiner.num_bn.bias', 'tabular_combiner.num_bn.running_mean', 'tabular_combiner.num_bn.running_var', 'tabular_combiner.num_bn.num_batches_tracked', 'tabular_combiner.g_cat_layer.weight', 'tabular_combiner.g_cat_layer.bias', 'tabular_combiner.h_cat_layer.weight', 'tabular_combiner.g_num_layer.weight', 'tabular_combiner.g_num_layer.bias', 'tabular_combiner.h_num_layer.weight', 'tabular_combiner.layer_norm.weight', 'tabular_combiner.layer_norm.bias', 'tabular_classifier.weight', 'tabular_classifier.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS

In [29]:
# create pipeline for text-classification (sentiment analysis)
pipe = pipeline(model=pretrained_model, task="sentiment-analysis", tokenizer=tokenizer)

predictions = []
# check labels of extracted tweets
for i, tweet in enumerate(tweets):
  print("Tweeet {}".format(i+1))
  prediction = pipe(tweet)
  
  # poner limite más alto?
  print(prediction[0]['score'])
  if prediction[0]['score'] >= 0.6:
    data_df.at[i,'label'] = 1
  else:
    data_df.at[i,'label'] = 0

Se han truncado las últimas 5000 líneas del flujo de salida.
Tweeet 38030
0.5846191048622131
Tweeet 38031
0.5203554034233093
Tweeet 38032
0.5274559855461121
Tweeet 38033
0.5403637886047363
Tweeet 38034
0.5698094964027405
Tweeet 38035
0.5697696805000305
Tweeet 38036
0.5226162075996399
Tweeet 38037
0.5426100492477417
Tweeet 38038
0.7157495021820068
Tweeet 38039
0.5053577423095703
Tweeet 38040
0.5432845950126648
Tweeet 38041
0.5497097373008728
Tweeet 38042
0.5258889198303223
Tweeet 38043
0.5190029740333557
Tweeet 38044
0.5502132177352905
Tweeet 38045
0.5756313800811768
Tweeet 38046
0.5332786440849304
Tweeet 38047
0.5707175135612488
Tweeet 38048
0.5933539271354675
Tweeet 38049
0.5523120164871216
Tweeet 38050
0.619707465171814
Tweeet 38051
0.5819442868232727
Tweeet 38052
0.5238878726959229
Tweeet 38053
0.5400456190109253
Tweeet 38054
0.5428959727287292
Tweeet 38055
0.5735872387886047
Tweeet 38056
0.5766487717628479
Tweeet 38057
0.5715335607528687
Tweeet 38058
0.5478449463844299
Tweeet 38059

In [30]:
num_tweets_hate = data_df['label'].tolist().count(1)
num_tweets_no_hate = data_df['label'].tolist().count(0)

print("Número de Tweets totales: {}".format(num_tweets_hate + num_tweets_no_hate))
print("Número de Tweets CON odio: {}".format(num_tweets_hate))
print("Número de Tweets SIN odio: {}".format(num_tweets_no_hate))
data_df.to_csv(data_path + 'all_csvs/todos_tweets_juntos_labeled.csv', index=False)

Número de Tweets totales: 40529
Número de Tweets CON odio: 6161
Número de Tweets SIN odio: 34368


In [ ]:
"""aciertos = 0
for i, pred in enumerate(predictions):
  if pred == data_df['label'].tolist()[i]:
    aciertos += 1

print(aciertos/len(predictions))"""


0.47360660572102625
